In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [2]:
train_dir ="data/train"
val_dir ="data/test"
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(120, 120),
    batch_size=64,
    color_mode="rgb",
    class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(120, 120),
    batch_size=64,
    color_mode="rgb",
    class_mode='categorical'
)

Found 791 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [3]:
models = Sequential()
models.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(120, 120, 3)))
models.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
models.add(MaxPooling2D(pool_size=(2, 2)))
models.add(Dropout(0.25))
models.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
models.add(MaxPooling2D(pool_size=(2, 2)))
models.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
models.add(MaxPooling2D(pool_size=(2, 2)))
models.add(Dropout(0.25))
models.add(Flatten())
models.add(Dense(1024, activation='relu'))
models.add(Dropout(0.5))
models.add(Dense(2, activation='softmax'))

c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
models.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001, decay=1e-6),
    metrics=['accuracy']
)

c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [5]:
models_info = models.fit(
    train_generator,
    steps_per_epoch=1800 // 64,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=1800 // 64
)


Epoch 1/40


c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/28 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.5036 - loss: 0.7238

c:\Users\nithi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 513ms/step - accuracy: 0.5101 - loss: 0.7166 - val_accuracy: 0.5300 - val_loss: 0.6866
Epoch 2/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 505ms/step - accuracy: 0.5842 - loss: 0.6806 - val_accuracy: 0.5800 - val_loss: 0.6808
Epoch 3/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 483ms/step - accuracy: 0.5581 - loss: 0.6770 - val_accuracy: 0.5400 - val_loss: 0.6783
Epoch 4/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 491ms/step - accuracy: 0.5735 - loss: 0.6742 - val_accuracy: 0.6000 - val_loss: 0.6611
Epoch 5/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 487ms/step - accuracy: 0.6262 - loss: 0.6430 - val_accuracy: 0.5900 - val_loss: 0.6473
Epoch 6/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 488ms/step - accuracy: 0.6570 - loss: 0.6217 - val_accuracy: 0.6200 - val_loss: 0.6320
Epoch 7/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 487ms/step - accuracy: 0.6480 - loss: 0.6158 - val_accuracy: 0.6100 - val_loss: 0.6218
Epoch 8/40
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 483ms/step - accuracy: 0.6630 - loss: 0.6082 - val_accuracy: 0.660

In [8]:
models.save_weights('model/model_weights.weights.h5')
model_json = models.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
with open('model/history.pckl', 'wb') as f:
    pickle.dump(models_info.history, f)
with open('model/history.pckl', 'rb') as f:
    data = pickle.load(f)


In [9]:
acc = data['accuracy']
accuracy = acc[-1] * 100
print("Training Model Accuracy = " + str(accuracy))

Training Model Accuracy = 95.70164084434509
